In [195]:
# importando bibliotecas
library('httr')
library("jsonlite")

In [196]:
# montando a carteira. Depois vamos pegar de algum lugar ou montar uma carteira mais real
carteiraJSON <- '[
  {"symbol" : "PETR4", "Position" : 1000, "Price" : 10}, 
  {"symbol" : "BBDC4", "Position" : 2000, "Price" : 5}
]' 

carteira <- fromJSON(carteiraJSON)

In [198]:
#pegando histórico de variação dos precos
query <- "http://api.informativos.io/prices_matrix?start_date=2018-08-13&end_date=2018-08-17&symbols[]=PETR4&symbols[]=BBDC4"
response <- content(GET(url=query), "text")

# transformando num dataframe
data <- as.data.frame(fromJSON(response, simplifyDataFrame = TRUE), stringsAsFactors=FALSE)

#colocando a primeira linha como cabecalho
normalize_data <- function(df) {
  names(df) <- as.character(unlist(df[1,]))
  df[-1,]
}
precos <- normalize_data(data)

precos_aplicados <- data.frame("Data" = c(), "Ativo" = c(), "Preco" = c(), stringsAsFactors = FALSE)

# iterando sobre a carteira
for (linha_carteira in 1:nrow(carteira)) {
    ativo <- carteira[linha_carteira, "symbol"]
    preco_original <- carteira[linha_carteira, "Price"]
    #iterando sobre os precos
    for (linha_preco in 2:nrow(precos)) {
        # Aplicando na carteira
        # primeiro preco, uso o preco original da carteira
        if (linha_preco == 2)
            preco_calculado = preco_original + sapply(precos[linha_preco, ativo], as.numeric)
        else
            preco_calculado = preco_calculado + sapply(precos[linha_preco -1, ativo], as.numeric)
        
        precos_aplicados <- rbind(precos_aplicados, data.frame("Data"=precos[linha_preco, "Date"], "Ativo"=ativo, "Preco"=preco_calculado))
    }    
}

# ajustando indices do dataframe
rownames(precos_aplicados) <- 1:nrow(precos_aplicados)

precos_aplicados

precos_aplicados[order(precos_aplicados$Preco),]



Data,Ativo,Preco
2018-08-14,PETR4,9.999501
2018-08-15,PETR4,9.999002
2018-08-16,PETR4,9.952455
2018-08-17,PETR4,9.947206
2018-08-14,BBDC4,5.017018
2018-08-15,BBDC4,5.034037
2018-08-16,BBDC4,5.021989
2018-08-17,BBDC4,5.021989


,Data,Ativo,Preco
5,2018-08-14,BBDC4,5.017018
7,2018-08-16,BBDC4,5.021989
8,2018-08-17,BBDC4,5.021989
6,2018-08-15,BBDC4,5.034037
4,2018-08-17,PETR4,9.947206
3,2018-08-16,PETR4,9.952455
2,2018-08-15,PETR4,9.999002
1,2018-08-14,PETR4,9.999501
